# Análises do Repositório de Dados Eleitorais do TSE sobre Receita e Despesa dos candidatos

### Perguntas a serem respondidas
    -Qual custo por voto de cada candidato em cada eleição?
    -Qual ranking de custo/voto dos candidatos
    -Qual custo por voto de candidatos do gênero feminino
    -Quais empresas com maior volume de doação
    -Quais empresas doaram para o maior número de candidatos
    -Qual volume de doações diretas e indiretas

In [1]:
import sqlite3
import unicodedata

import matplotlib
import pandas

## Importa dados de Receita e Despesa

# Eleições - 2016

In [4]:
YEAR = 2016

In [2]:
conn = sqlite3.connect('../receitas-despesas-votacao.sqlite')

In [3]:
receita = pandas.read_sql(
    f"""SELECT SUM(valor_receita) as receita_total, nome_candidato
    FROM receita WHERE ano_eleicao = '{YEAR}' AND cpf_candidato <> ''
    GROUP BY cpf_candidato ORDER BY receita_total DESC LIMIT 10000""",
    conn
)

despesa = pandas.read_sql(
    f"""SELECT SUM(valor_despesa) as despesa_total, nome_candidato
    FROM despesa WHERE ano_eleicao = '{YEAR}' AND cpf_candidato <> ''
    GROUP BY cpf_candidato ORDER BY despesa_total DESC LIMIT 10000""",
    conn
)

## 10000 Candidatos com maior receita

In [5]:
receita.head()

,receita_total,nome_candidato
0,40000070.00,VAGNER FORTUNATO SOARES
1,17271197.02,ERALDO PAIXAO DOS SANTOS
2,12446819.95,JOAO AGRIPINO DA COSTA DORIA JUNIOR
3,10407826.88,ROBERTO CLAUDIO RODRIGUES BEZERRA
4,9664174.81,MARCELO BEZERRA CRIVELLA


## 10000 Candidatos com maior despesa

In [6]:
despesa.head()

,despesa_total,nome_candidato
0,43222309.02,ERALDO PAIXAO DOS SANTOS
1,40000070.00,VAGNER FORTUNATO SOARES
2,16212629.19,FERNANDO HADDAD
3,12443026.13,JOAO AGRIPINO DA COSTA DORIA JUNIOR
4,10374162.27,ROBERTO CLAUDIO RODRIGUES BEZERRA


## Soma dos votos considerando ambos os turnos

In [12]:
votos = pandas.read_sql(
    f"""SELECT NOME, sigla_uf, SUM(total_votos) AS votos
    FROM votacao_zona
    WHERE ano_eleicao = {YEAR} GROUP BY NOME
    ORDER BY votos DESC LIMIT 10000;""",
    conn
)

In [13]:
votos.head()

,nome,sigla_uf,votos
0,MARCELO BEZERRA CRIVELLA,RJ,2542231
1,MARCELO RIBEIRO FREIXO,RJ,1717086
2,ANTONIO CARLOS PEIXOTO DE MAGALHAES NETO,BA,982246
3,GERALDO JULIO DE MELLO FILHO,PE,959332
4,JOAO LEITE DA SILVA NETO,MG,953308


## Inner Join de votos com receita

In [44]:
votos_receita = receita.merge(votos, left_on='nome_candidato', right_on='nome', how='inner')

## Inner Join de votos com despesa

In [45]:
votos_despesa = despesa.merge(votos, left_on='nome_candidato', right_on='nome', how='inner')

## Receita por voto

In [46]:
votos_receita['receita_por_voto'] = votos_receita.votos / votos_receita.receita_total

In [47]:
votos_receita.head()

,receita_total,nome_candidato,nome,sigla_uf,votos,receita_por_voto
0,9664174.81,MARCELO BEZERRA CRIVELLA,MARCELO BEZERRA CRIVELLA,RJ,2542231,0.263057
1,9223254.27,PEDRO PAULO CARVALHO TEIXEIRA,PEDRO PAULO CARVALHO TEIXEIRA,RJ,488775,0.052994
2,8351508.69,ANTONIO CARLOS PEIXOTO DE MAGALHAES NETO,ANTONIO CARLOS PEIXOTO DE MAGALHAES NETO,BA,982246,0.117613
3,5789176.32,RODRIGO OTAVIO SOARES PACHECO,RODRIGO OTAVIO SOARES PACHECO,MG,118772,0.020516
4,4792885.30,CARLOS ENRIQUE FRANCO AMASTHA,CARLOS ENRIQUE FRANCO AMASTHA,TO,68634,0.014320


## Despesa por voto

In [48]:
votos_despesa['despesa_por_voto'] = votos_despesa.votos / votos_despesa.despesa_total

In [116]:
votos_despesa.head()

,despesa_total,nome_candidato,nome,sigla_uf,votos,despesa_por_voto,names_normalized,gender_classification
0,10127632.02,JOAO LEITE DA SILVA NETO,JOAO LEITE DA SILVA NETO,MG,953308,0.094129,JOAO,Masculino
1,9904939.51,ANTONIO CARLOS PEIXOTO DE MAGALHAES NETO,ANTONIO CARLOS PEIXOTO DE MAGALHAES NETO,BA,982246,0.099167,ANTONIO,Masculino
2,9707448.16,PEDRO PAULO CARVALHO TEIXEIRA,PEDRO PAULO CARVALHO TEIXEIRA,RJ,488775,0.050351,PEDRO,Masculino
3,9628571.15,MARCELO BEZERRA CRIVELLA,MARCELO BEZERRA CRIVELLA,RJ,2542231,0.264030,MARCELO,Masculino
4,6848430.17,RODRIGO OTAVIO SOARES PACHECO,RODRIGO OTAVIO SOARES PACHECO,MG,118772,0.017343,RODRIGO,Masculino


# Pensar em algumas visualizações

## Load model to predict gender based on first name

In [49]:
import pickle

import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

## Function to extract features from name

In [50]:
@np.vectorize
def features(name):
    name = name.lower()
    return {
        'first-letter': name[0],
        'first2-letters': name[:2],
        'first3-letters': name[:3],
        'last-letter': name[-1],
        'last2-letters': name[-2:],
        'last3-letters': name[-3:]
    }




In [51]:
model = pickle.load(open('modelo_genero_nome.pkl', 'rb'))
vectorizer = pickle.load(open('vectorizer_genero_nome.pkl', 'rb'))

## Apply classification

In [52]:
@np.vectorize
def classify(name, vectorizer):
    name = name.lower()
    name_transformed = vectorizer.transform(features([name]))
    return 'Masculino' if model.predict(name_transformed) == 1 else 'Feminino'

## Import table with names

In [53]:
names = pandas.read_csv('classificacao_nomes.csv', sep=';')

In [94]:
@np.vectorize
def classify_db(name):
    if name not in names.first_name.values:
        return 0
    class_ = names.loc[names.first_name == name].classification.values[0]
    return 'Masculino' if class_ else 'Feminino'

## Classify names using list from brazilian database of names

In [91]:
def remove_accent(name):
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode().upper()

In [106]:
votos_receita['names_normalized'] = votos_receita.nome_candidato.map(lambda x: remove_accent(x.split(' ')[0]))
votos_despesa['names_normalized'] = votos_despesa.nome_candidato.map(lambda x: remove_accent(x.split(' ')[0]))

votos_receita['gender_classification'] = classify_db(votos_receita.names_normalized)
votos_despesa['gender_classification'] = classify_db(votos_despesa.names_normalized)

## Classify Gender using first name and Logistic Regression model of candidates that the names is not in the DB

In [107]:
votos_despesa.loc[votos_despesa.gender_classification == '0', 'gender_classification'] = classify(
    votos_despesa.loc[votos_despesa.gender_classification == '0']['names_normalized'], vectorizer)

votos_receita.loc[votos_receita.gender_classification == '0', 'gender_classification'] = classify(
    votos_receita.loc[votos_receita.gender_classification == '0']['names_normalized'], vectorizer)

### Correct most evident missclassification

    - José (Jose)
    - SCHAMBERLAEN

In [109]:
missclassified_names = ['JOSE', 'SCHAMBERLAEN']

In [110]:
votos_despesa.loc[votos_despesa.names_normalized.isin(missclassified_names), 
                  'gender_classification'] = 'Masculino'
votos_receita.loc[votos_receita.names_normalized.isin(missclassified_names),
                  'gender_classification'] = 'Masculino'

In [111]:
votos_despesa.loc[votos_despesa.gender_classification == 'Feminino'].sort_values(['votos', 'despesa_por_voto'])

,despesa_total,nome_candidato,nome,sigla_uf,votos,despesa_por_voto,names_normalized,gender_classification
4310,68543.06,ADRIANE BORTOLASO SCHRAMM,ADRIANE BORTOLASO SCHRAMM,RS,2091,0.030506,ADRIANE,Feminino
5233,55356.88,LUANA ADRIANA DA SILVA TORRES,LUANA ADRIANA DA SILVA TORRES,MG,2095,0.037845,LUANA,Feminino
3609,80928.75,CELIA SILVEIRA DE MORAIS ALMEIDA,CELIA SILVEIRA DE MORAIS ALMEIDA,GO,2097,0.025912,CELIA,Feminino
5132,56699.80,JACIRA MARTINS FERNANDES,JACIRA MARTINS FERNANDES,GO,2112,0.037249,JACIRA,Feminino
4336,67990.19,EDNEA BUCHI BATISTA,EDNEA BUCHI BATISTA,PR,2116,0.031122,EDNEA,Feminino
5068,57597.40,MARIA DO SOCORRO RODRIGUES ALVES BRAGA,MARIA DO SOCORRO RODRIGUES ALVES BRAGA,RO,2121,0.036825,MARIA,Feminino
4408,66830.80,FERNANDA SOARES PENA FERRAZ,FERNANDA SOARES PENA FERRAZ,MG,2124,0.031782,FERNANDA,Feminino
1938,128175.89,ZILDA DE REZENDE ARAUJO MACHADO,ZILDA DE REZENDE ARAUJO MACHADO,GO,2126,0.016587,ZILDA,Feminino
5481,52158.46,SANDRA MARIA DA ESCOSSIA ROSADO,SANDRA MARIA DA ESCOSSIA ROSADO,RN,2129,0.040818,SANDRA,Feminino
4861,60480.06,ELIETE ALVES DE MELO,ELIETE ALVES DE MELO,TO,2141,0.035400,ELIETE,Feminino


In [112]:
votos_receita.loc[votos_receita.gender_classification == 'Feminino'].sort_values('receita_total', ascending=False)

,receita_total,nome_candidato,nome,sigla_uf,votos,receita_por_voto,names_normalized,gender_classification
6,4228071.16,ROSIANE MODESTO DE OLIVEIRA,ROSIANE MODESTO DE OLIVEIRA,MS,283398,0.067028,ROSIANE,Feminino
31,1853838.50,IRIS REZENDE MACHADO,IRIS REZENDE MACHADO,GO,656392,0.354072,IRIS,Feminino
34,1701023.17,MARIA TERESA SAENZ SURITA GUIMARAES,MARIA TERESA SAENZ SURITA GUIMARAES,RR,121148,0.071221,MARIA,Feminino
42,1486830.00,ANGELA REGINA HEINZEN AMIN HELOU,ANGELA REGINA HEINZEN AMIN HELOU,SC,171749,0.115514,ANGELA,Feminino
46,1393229.00,MARIA APARECIDA RAMOS DE MENESES,MARIA APARECIDA RAMOS DE MENESES,PB,125146,0.089824,MARIA,Feminino
56,1297000.00,MARIA VICTORIA BORGHETTI BARROS,MARIA VICTORIA BORGHETTI BARROS,PR,52576,0.040537,MARIA,Feminino
74,1162430.00,LAUREZ DA ROCHA MOREIRA,LAUREZ DA ROCHA MOREIRA,TO,20640,0.017756,LAUREZ,Feminino
82,1106117.92,LUCIA ELISABETH COLOMBO SILVEIRA,LUCIA ELISABETH COLOMBO SILVEIRA,RS,152215,0.137612,LUCIA,Feminino
89,1037770.00,CLAUDIA TELLES DE MENEZES PIRES MARTINS LELIS,CLAUDIA TELLES DE MENEZES PIRES MARTINS LELIS,TO,13121,0.012643,CLAUDIA,Feminino
92,1023000.00,PRISCILA KRAUSE BRANCO,PRISCILA KRAUSE BRANCO,PE,47399,0.046333,PRISCILA,Feminino


In [115]:
pandas.read_sql('SELECT * FROM votacao_zona LIMIT 1;', conn).columns

Index(['ano_eleicao', 'codigo_eleicao', 'codigo_tipo_eleicao', 'data_eleicao',
       'descricao_eleicao', 'nome_tipo_eleicao', 'tipo_abrangencia_eleicao',
       'codigo_totalizacao_turno', 'descricao_totalizacao_turno',
       'numero_turno', 'codigo_municipio', 'descricao_ue', 'nome_municipio',
       'sigla_ue', 'sigla_uf', 'codigo_legenda', 'composicao_legenda',
       'nome_coligacao', 'nome_legenda', 'nome_partido', 'numero_partido',
       'numero_sequencial_legenda', 'sigla_partido', 'codigo_cargo',
       'codigo_detalhe_situacao_candidatura', 'codigo_situacao_candidatura',
       'codigo_situacao_candidatura_superior', 'descricao_cargo',
       'descricao_detalhe_situacao_candidatura',
       'descricao_situacao_candidatura',
       'descricao_situacao_candidatura_superior', 'nome', 'nome_social',
       'nome_urna', 'numero_sequencial', 'numero_urna', 'tipo_agremiacao',
       'numero_zona', 'total_votos', 'voto_em_transito'],
      dtype='object')